<a href="https://colab.research.google.com/github/BhaveshGoswami11/Generative-AI/blob/main/Generative_AI_chatbot_using_LLM_from_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Chatbot Tutorial - Using Local Models (No RAG, No API)

**Quick Start**: Click the "Open in Colab" button above to run this notebook in Google Colab!

---

## 1) Introduction & Objectives

### What is a Simple Chatbot?
A simple chatbot uses a pre-trained language model that runs locally without needing external APIs or retrieval systems. Unlike RAG chatbots that search external knowledge bases, these chatbots rely entirely on the knowledge encoded in the model during training.

### Key Differences from RAG:
- **No external knowledge**: Answers come only from the model's training data
- **No API costs**: Uses free, open-source models from Hugging Face
- **Faster setup**: No need to create and maintain a vector database
- **Limited knowledge**: Cannot access information after the model's training cutoff date
- **Risk of hallucinations**: May generate incorrect information without source verification

### Use Cases:
- Simple Q&A bots for general knowledge
- Conversational interfaces for learning
- Prototyping chatbot interfaces
- Understanding basic LLM behavior

## 2) Install Dependencies

We'll use Hugging Face's Transformers library to load a pre-trained conversational model.

### Libraries and their purposes:
- **transformers** → Provides access to thousands of pre-trained models from Hugging Face
- **torch** → PyTorch framework needed to run neural networks
- **ipywidgets** → Creates interactive UI elements in Google Colab

In [ ]:
!pip install -q transformers torch ipywidgets accelerate

## 3) Load a Pre-trained Model

We'll use **TinyLlama-1.1B-Chat** - a small but capable model that runs well in Colab.

### Why TinyLlama?
- Small size (1.1B parameters) - runs fast in free Colab
- Designed for conversations
- No API key required
- Good balance of quality and speed

### Key Concepts:
- **Tokenizer**: Converts text into numbers the model understands
- **Model**: The neural network that generates responses
- **Generation settings**: Control how creative or focused the responses are

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("🔄 Loading model...")

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Fast & good quality

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Model loaded: {model_name}")

## 4) Create Chat Function

This function handles the conversation logic:

### How it works:
1. **Encode input**: Convert user's text into tokens
2. **Generate response**: Model predicts the next tokens
3. **Decode output**: Convert tokens back to readable text
4. **Maintain context**: Keep conversation history for coherent multi-turn chats

### Parameters explained:
- **max_length**: Maximum total tokens (input + output)
- **temperature**: Controls randomness (0.7 = balanced, higher = more creative)
- **top_p**: Nucleus sampling - considers top 90% probable words
- **do_sample**: Enables random sampling for varied responses
- **pad_token_id**: Tells model how to handle padding

In [ ]:
def chat_with_bot(user_input, conversation_history):

    # Build conversation context
    prompt = ""
    for turn in conversation_history[-6:]:  # Keep last 6 turns for context
        if turn['role'] == 'user':
            prompt += f"User: {turn['text']}\n"
        else:
            prompt += f"Assistant: {turn['text']}\n"

    prompt += f"User: {user_input}\nAssistant:"

    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate with better parameters
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,  # Limit response length
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2  # Reduce repetition
        )

    # Decode only the new tokens
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

    # Clean up response
    response = response.split("User:")[0].strip()  # Remove any follow-up prompts
    response = response.split("\n\n")[0].strip()   # Take first paragraph

    return response

## 5) Test the Chatbot (Simple Version)

Before building the UI, let's test the chatbot with a few example questions.

In [ ]:
# Initialize conversation
chat_history = [] # Initialize as an empty list

# Test conversation
print("=== Testing Chatbot ===\n")

test_questions = [
    "Hello! How are you?",
    "What do you like to talk about?",
    "Tell me a fun fact"
]

for question in test_questions:
    print(f"You: {question}")
    response = chat_with_bot(question, chat_history)
    print(f"Bot: {response}\n")

print("✅ Test complete! Now let's build the interactive UI.")

## 6) Interactive Chat UI

Now we'll create a user-friendly interface similar to the RAG chatbot, but simpler since we don't need retrieval.

### UI Components:
- **Input box**: Where users type their messages
- **Send button**: Sends the message to the bot
- **Clear button**: Resets the conversation
- **Quit button**: Closes the chat interface
- **Chat display**: Shows the conversation history

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

input_box = widgets.Text(
    placeholder='Type your message...',
    description='You:',
    layout=widgets.Layout(width='80%')
)
send_button = widgets.Button(description='Send', button_style='primary')
clear_button = widgets.Button(description='Clear', button_style='warning')
quit_button = widgets.Button(description='Quit', button_style='danger')
output = widgets.HTML(value='')

conversation_history = []

def render_chat():
    html = '<div style="max-height: 400px; overflow-y: auto; border: 1px solid #ccc; padding: 10px; border-radius: 5px; background-color: #f9f9f9;">'

    if not conversation_history:
        html += '<p style="color: #666; text-align: center;">Start chatting below! 👇</p>'

    for turn in conversation_history:
        role = turn['role']
        text = turn['text'].replace('\n', '<br>')

        if role == 'user':
            html += f'''
            <div style="margin: 10px 0; text-align: right;">
                <span style="background-color: #007bff; color: white; padding: 10px 15px; border-radius: 18px; display: inline-block; max-width: 70%; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
                    {text}
                </span>
            </div>
            '''
        else:
            html += f'''
            <div style="margin: 10px 0;">
                <span style="background-color: #ffffff; color: black; padding: 10px 15px; border-radius: 18px; display: inline-block; max-width: 70%; box-shadow: 0 2px 5px rgba(0,0,0,0.1); border: 1px solid #e0e0e0;">
                    {text}
                </span>
            </div>
            '''

    html += '</div>'
    output.value = html

def on_send_clicked(b):
    user_message = input_box.value.strip()
    if not user_message:
        return

    # Add user message
    conversation_history.append({'role': 'user', 'text': user_message})
    render_chat()
    input_box.value = ''

    # Show thinking indicator
    conversation_history.append({'role': 'bot', 'text': '💭 Thinking...'})
    render_chat()

    try:
        # Get bot response
        bot_response = chat_with_bot(user_message, conversation_history[:-1])

        # Remove thinking indicator and add real response
        conversation_history.pop()
        conversation_history.append({'role': 'bot', 'text': bot_response})
        render_chat()
    except Exception as e:
        conversation_history.pop()
        conversation_history.append({'role': 'bot', 'text': f'❌ Error: {str(e)}'})
        render_chat()

def on_clear_clicked(b):
    conversation_history.clear()
    render_chat()

def on_quit_clicked(b):
    clear_output(wait=True)
    print("Chat closed. Re-run this cell to restart.")

send_button.on_click(on_send_clicked)
clear_button.on_click(on_clear_clicked)
quit_button.on_click(on_quit_clicked)

ui = widgets.VBox([
    widgets.HTML(f'<h3>🤖 Improved Chatbot</h3><p style="color: #666;">Using: {model_name}</p>'),
    output,
    input_box,
    widgets.HBox([send_button, clear_button, quit_button])
])

display(ui)
render_chat()
print("💬 Chat ready!")

## 7) Comparison: RAG vs Simple Chatbot

### RAG Chatbot (What we built before):
✅ Accesses external knowledge sources  
✅ Can answer domain-specific questions accurately  
✅ Provides source citations  
✅ Stays up-to-date with current information  
❌ Requires API keys and costs money  
❌ Slower due to retrieval step  
❌ More complex setup  

### Simple Chatbot (What we built now):
✅ No API costs - completely free  
✅ Simpler setup  
✅ Good for general conversation  
❌ Limited to training data knowledge  
❌ Can't access specific/recent information  
❌ No source verification  
❌ Higher risk of hallucinations

## 8) Key Takeaways

Students should understand:

1. **Simple chatbots** are good for general conversation but lack domain expertise
2. **RAG chatbots** excel when specific, accurate information is needed
3. **API-based models** (like GPT-4) are more powerful but cost money
4. **Open-source models** are free but have limitations
5. **Trade-offs** exist between cost, speed, accuracy, and complexity

### When to use each approach:
- **Simple Chatbot**: Personal projects, learning, prototyping, general chat
- **RAG Chatbot**: Business applications, customer support, knowledge bases
- **No Chatbot Needed**: When simple rules or search would work better!